# Description

In this notebook we explore how skeleton data looks. In particular, we need to test whether we can generate satisfying smoothing for the skeleton data that doesn't impair time resolution

# Imports

In [ ]:
from pathlib import Path
import sys

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from scipy.signal import correlate
from sklearn.metrics import mean_squared_error
import holoviews as hv
hv.extension('bokeh')
from scipy.signal import savgol_filter

from pykalman import KalmanFilter

src_path = Path.cwd().parent / "src"
if src_path not in sys.path:
    sys.path.append(str(src_path))

from Ballpushing_utils import Experiment, Fly

from utils_behavior import Processing

from utils_behavior import Sleap_utils

In [ ]:
ExampleFly_path = Path(
    "/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231130_TNT_Fine_2_Videos_Tracked/arena2/corridor5"
)

ExampleFly = Fly(ExampleFly_path, as_individual=True)

In [ ]:
skeleton_data = ExampleFly.tracking_data.fly_skeleton

In [ ]:
skeleton_data

# Functions to test smoothing parameters

Here we're gonna test the mean squared error and the time lag between smoothed and raw data to find the best smoothing parameter that reduce the tracking error-related noise while keeping time resolution.

In [ ]:
def compute_time_lag(raw_data, smoothed_data, sampling_rate):
    correlation = correlate(smoothed_data, raw_data, mode="full")
    lag = correlation.argmax() - (len(raw_data) - 1)
    time_lag = lag / sampling_rate  # Convert lag to time
    return time_lag

In [ ]:
def compute_error(raw_data, smoothed_data):
    mse = mean_squared_error(raw_data, smoothed_data)
    return mse

In [ ]:
def test_filter_parameters(raw_data, sampling_rate, window_lengths, polyorders):
    results = []
    for wl in window_lengths:
        for po in polyorders:
            try:
                smoothed_data = Processing.savgol_lowpass_filter(raw_data, window_length=wl, polyorder=po)
                time_lag = compute_time_lag(raw_data, smoothed_data, sampling_rate)
                mse = compute_error(raw_data, smoothed_data)
                results.append((wl, po, time_lag, mse))
            except ValueError:
                print(f"Invalid parameters: window_length={wl}, polyorder={po}")
                continue  # Skip invalid parameter combinations
    return results

# Tests

In [ ]:
# Define the parameters to test
window_lengths = [11, 21, 51, 101, 151, 201, 251, 301]  # Must be odd integers
polyorders = [1,]
sampling_rate = ExampleFly.experiment.fps  # Replace with the actual sampling rate of your data

# Initialize a dictionary to store results for each keypoint
results = {}

# Iterate over each keypoint column in the skeleton_data
for column in skeleton_data.columns:
    if column.startswith("x_") or column.startswith("y_"):  # Filter keypoint columns
        raw_data = skeleton_data[column].dropna().values  # Remove NaN values
        if len(raw_data) < min(window_lengths):  # Skip if data is too short
            print(f"Skipping {column} as data length={len(raw_data)} is too short")
            continue

        # Test filter parameters for the current keypoint
        keypoint_results = test_filter_parameters(raw_data, sampling_rate, window_lengths, polyorders)

        # Store the results
        results[column] = keypoint_results

# Display results for each keypoint
for keypoint, keypoint_results in results.items():
    print(f"Results for {keypoint}:")
    for wl, po, time_lag, mse in keypoint_results:
        print(f"  Window Length: {wl}, Polyorder: {po}, Time Lag: {time_lag:.4f}s, MSE: {mse:.4f}")

In [ ]:
# Define a function to find the best parameters
def find_best_parameters(results):
    best_parameters = {}
    for keypoint, keypoint_results in results.items():
        # Sort by MSE and then by time lag to prioritize low error and minimal lag
        sorted_results = sorted(keypoint_results, key=lambda x: (x[3], abs(x[2])))  # Sort by MSE, then time lag
        best_parameters[keypoint] = sorted_results[0]  # Take the best combination
    return best_parameters


# Find the best parameters for each keypoint
best_parameters = find_best_parameters(results)

# Display the best parameters for each keypoint
print("Best parameters for each keypoint:")
for keypoint, (wl, po, time_lag, mse) in best_parameters.items():
    print(f"{keypoint}: Window Length={wl}, Polyorder={po}, Time Lag={time_lag:.4f}s, MSE={mse:.4f}")

# Optionally, find the overall best parameters across all keypoints
overall_best = min(
    [(keypoint, wl, po, time_lag, mse) for keypoint, (wl, po, time_lag, mse) in best_parameters.items()],
    key=lambda x: (x[4], abs(x[3])),  # Sort by MSE, then time lag
)

print("\nOverall best parameters across all keypoints:")
print(
    f"Keypoint={overall_best[0]}, Window Length={overall_best[1]}, Polyorder={overall_best[2]}, "
    f"Time Lag={overall_best[3]:.4f}s, MSE={overall_best[4]:.4f}"
)

In [ ]:
skeletontrack = Sleap_utils.Sleap_Tracks(
    ExampleFly_path / "corridor5_preprocessed_full_body.h5", smoothed_tracks=True, smoothing_params=(11, 3)
)

skeletontrack.video

In [ ]:
skeletontrack = Sleap_utils.Sleap_Tracks(ExampleFly_path/"corridor5_preprocessed_full_body.h5", smoothed_tracks=True, smoothing_params=(11,1)).objects[0].dataset

In [ ]:
skeletontrack

In [ ]:
def calculate_distances(data, reference_key):
    """Calculate distances between each keypoint and the reference keypoint."""
    distances = {}
    ref_x, ref_y = data[f"x_{reference_key}"], data[f"y_{reference_key}"]
    for column in data.columns:
        if column.startswith("x_") and column != f"x_{reference_key}":
            key = column[2:]  # Extract keypoint name
            key_x, key_y = data[f"x_{key}"], data[f"y_{key}"]
            distances[key] = np.sqrt((key_x - ref_x) ** 2 + (key_y - ref_y) ** 2)
    return pd.DataFrame(distances)


def bootstrap_ci(data, n_bootstrap=1000, ci=95):
    """Calculate bootstrapped confidence intervals for each column."""
    lower_bound = (100 - ci) / 2
    upper_bound = 100 - lower_bound
    ci_bounds = {}
    for column in data.columns:
        bootstrap_samples = [
            np.mean(np.random.choice(data[column].dropna(), size=len(data[column]), replace=True))
            for _ in range(n_bootstrap)
        ]
        ci_bounds[column] = (
            np.percentile(bootstrap_samples, lower_bound),
            np.percentile(bootstrap_samples, upper_bound),
        )
    return ci_bounds


def detect_outliers(distances, ci_bounds):
    """Detect outliers based on confidence intervals."""
    outliers = pd.DataFrame(False, index=distances.index, columns=distances.columns)
    for key, (lower, upper) in ci_bounds.items():
        outliers[key] = (distances[key] < lower) | (distances[key] > upper)
    return outliers

def smooth_outliers(data, outliers, window=3):
    """Smooth out outliers using neighboring frames."""
    smoothed_data = data.copy()
    for column in outliers.columns:
        for idx in outliers.index[outliers[column]]:
            # Replace outlier with the mean of neighboring frames
            start = max(0, idx - window)
            end = min(len(data), idx + window + 1)
            smoothed_data.loc[idx, column] = data.loc[start:end, column].mean()
    return smoothed_data

In [ ]:
reference_key = "Thorax"  # Use Thorax as the reference point
distances = calculate_distances(skeleton_data, reference_key)
ci_bounds = bootstrap_ci(distances)
outliers = detect_outliers(distances, ci_bounds)
smoothed_distances = smooth_outliers(distances, outliers)

In [ ]:
smoothed_data = skeleton_data.copy()

for key in distances.columns:
    x_col, y_col = f"x_{key}", f"y_{key}"
    smoothed_data.loc[outliers[key], [x_col, y_col]] = np.nan  # Mark outliers as NaN
    smoothed_data[[x_col, y_col]] = smoothed_data[[x_col, y_col]].interpolate()  # Interpolate missing values

# Looking at a small subset of data

Will be useful in order to compare smoothed and raw data

In [ ]:
# Plot sns curves of smoothed and raw data for a specific keypoint across 10 seconds

def plot_keypoint_data(raw_data, smoothed, keypoint, start_time=0, end_time=10):
    # Extract the x and y coordinates for the specified keypoint
    raw_data = skeleton_data[f"y_{keypoint}"].dropna().values
    smoothed_data = smoothed_data[f"y_{keypoint}"].dropna().values

    # crop the smoothed data to the same length as the raw data
    smoothed_data = smoothed_data[:len(raw_data)]

    # Create a time array based on the sampling rate
    time = np.arange(len(raw_data)) / ExampleFly.experiment.fps

    # Filter the data for the specified time range
    mask = (time >= start_time) & (time <= end_time)
    time = time[mask]
    raw_data = raw_data[mask]
    smoothed_data = smoothed_data[mask]

    # Plot the data using seaborn
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=time, y=raw_data, label='Y Coordinate', color='blue')
    sns.lineplot(x=time, y=smoothed_data, label='Y Coordinate_smoothed', color='orange')
    plt.title(f"Keypoint Data for {keypoint} from {start_time}s to {end_time}s")
    plt.xlabel("Time (s)")
    plt.ylabel("Coordinate Value")
    plt.legend()
    plt.show()

In [ ]:
# Function to plot keypoint data for a specific time range
def plot_keypoint_data(raw_data, smoothed, keypoint, start_time, end_time):
    # Extract the y-coordinates for the specified keypoint
    raw_data = raw_data[f"y_{keypoint}"].dropna().values
    smoothed_data = smoothed[f"y_{keypoint}"].dropna().values

    # Crop the smoothed data to the same length as the raw data
    smoothed_data = smoothed_data[: len(raw_data)]

    # Create a time array based on the sampling rate
    time = np.arange(len(raw_data)) / ExampleFly.experiment.fps

    # Filter the data for the specified time range
    mask = (time >= start_time) & (time <= end_time)
    time = time[mask]
    raw_data = raw_data[mask]
    smoothed_data = smoothed_data[mask]

    # Plot the data using seaborn
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=time, y=raw_data, label="Y Coordinate (Raw)", color="blue")
    sns.lineplot(x=time, y=smoothed_data, label="Y Coordinate (Smoothed)", color="orange")
    plt.title(f"Keypoint Data for {keypoint} from {start_time:.2f}s to {end_time:.2f}s")
    plt.xlabel("Time (s)")
    plt.ylabel("Coordinate Value")
    plt.legend()
    plt.show()


# Sample 20 random bouts and plot
Key = "Lfront"
video_duration = len(skeleton_data) / ExampleFly.experiment.fps  # Total video duration in seconds
bout_duration = 10  # Duration of each bout in seconds

for i in range(20):
    start_time = random.uniform(0, video_duration - bout_duration)
    end_time = start_time + bout_duration
    print(f"Plotting bout {i+1}: {start_time:.2f}s to {end_time:.2f}s")
    plot_keypoint_data(skeleton_data, smoothed_data, Key, start_time, end_time)

## Saving the smoothed test 

# Generating a Hv curve

Holoviews allow precise zooming so we'll use it to visually explore the smoothed vs raw data

In [ ]:
# Plot the raw and smoothed data for a specific keypoint

Key = "Lfront"

raw_data = skeleton_data[f"x_{Key}"].dropna().values
smoothed_data = skeletontrack[f"x_{Key}"].dropna().values
# crop the smoothed_data to match the length of raw_data
smoothed_data = smoothed_data[: len(raw_data)]
time = np.arange(len(raw_data)) / ExampleFly.experiment.fps  # Time vector

Curves = hv.Curve((time, raw_data), label="Raw Data").opts(
    title=f"Raw and Smoothed Data for {Key}",
    xlabel="Time (s)",
    ylabel="Position",
    color="blue",
    width=800,
    height=400,
)
smoothed_curve = hv.Curve((time, smoothed_data), label="Smoothed Data").opts(
    color="red",
    width=800,
    height=400,
)
# Overlay the curves

overlay = Curves * smoothed_curve

# Save the plot as html
hv.save(overlay, "/mnt/upramdya_data/MD/Pictures/smoothed_data_plot.html", backend="bokeh")

# Generating a labelled clip for tracking

In [ ]:
Sleap_utils.generate_annotated_video(
    video=skeletontrack.video,
    sleap_tracks_list=[skeletontrack],
    output_path=ExampleFly_path / "Annotated_Video.mp4",
)